In [151]:
import numpy as np
from PIL import Image,ImageOps
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [155]:

def convertImageToArrayRGB(imagefile, grayscale):
    """
    Input: Image .JPG file
    Output: 2D array representing the image.
    """
    imageObject = Image.open(imagefile)
    if not grayscale :
        return np.array(imageObject)
    else :
        gray_image = ImageOps.grayscale(imageObject)
        return np.array(gray_image)
    

img_2 = convertImageToArrayRGB('L4.jpg', False)
img_2_Gray = convertImageToArrayRGB('L4.jpg', True)
img_2_Gray


array([[27, 27, 27, ..., 27, 27, 27],
       [27, 28, 27, ..., 27, 27, 27],
       [27, 28, 27, ..., 27, 27, 27],
       ...,
       [32, 35, 35, ..., 40, 40, 39],
       [32, 35, 35, ..., 38, 39, 40],
       [32, 34, 35, ..., 38, 38, 39]], dtype=uint8)

In [159]:
def CalculateIntegral(arrayImage):
    """
    Input: 2D array representing the image (feel free to use a predefined function to transform an image into an array).
    Output: 2D array representing the integral image.
    """
    height = len(arrayImage)
    width = len(arrayImage)

    #Init a 3d array of length, width and RGB values
    si = np.zeros(arrayImage.shape)
    si.astype(int)
    si.setflags(write=1)

    #Add First Coloumn to SI
    for i in range(0,height):
        si[i][0] =  arrayImage[i][0]

    #Get SI    
    for i in range(0,height):
        for j in range(1,width):
            si[i][j] = arrayImage[i][j] + si[i][j-1]
    #Get II
    for j in range(0,width):
        for i in range(1,height):
            si[i][j] = si[i][j] + si[i-1][j]       
    
    return si

In [162]:
si = CalculateIntegral(img_2)
si_g = CalculateIntegral(img_2_Gray)
print(si_g[600][80])

1340611.0


In [163]:
def CalculateLocalSum(intergralImage, left_corner, right_corner):
    """
    Input: An integral image, and a two pairs of coordinates (𝑝0=(𝑥0,𝑦0),𝑝1=(𝑥1,𝑦1)).
    Output: The local sum for the rectangular area defined by the pair of points (as 𝑝0 being the upper left corner, and 𝑝1 being the lower right corner of this                  rectangular area).
    """
    # Returns Intensity Relative to Origin
    if(left_corner==0 and right_corner!=-1):
        return intergralImage[right_corner[0]][right_corner[1]] - intergralImage[0][right_corner[1]] - intergralImage[right_corner[0]][0] + intergralImage[0][0]
    #Returns Difference between right and left indicies.
    else:
        return intergralImage[right_corner[0]][right_corner[1]] - intergralImage[left_corner[0]][left_corner[1]]
        


print(CalculateLocalSum(si,(300,300),(500,500)))
print(CalculateLocalSum(si_g,0,(500,500)))

[3280464. 4737887. 6180312.]
6882971.0


In [164]:
def kernelGenerator(size,kernelType):
    """
    Input: The size of the Kernel and Its Type
    Output: returns the requested kernel and describes it breifly
    """
    if(size%2 ==0 or size ==1):
        print("Size has to be an Odd Number >1")
        return

    middle_row = math.floor(size/2)    
    if(kernelType.lower()=="prewitt"):
        print("Returning Prewitt of Size: %s" %size)

        #Computing Value of H1
        h1 = np.zeros(size**2).reshape(size,size)
        h1[0:middle_row] = [1]*size
        h1[middle_row] = [0]*size
        h1[middle_row+1:] = [-1]*size

        #Computing value of H2
        h2 = np.copy(h1)
        h2 = np.transpose(h2)

        return h1,h2

    elif(kernelType.lower()=="laplace"):
        print("Returning Laplace of Size: %s "%size)
        h1 = np.zeros(size**2).reshape(size,size) - 1
        h1[middle_row][middle_row] = (size**2) -1
        return h1,[]

    elif(kernelType.lower()=="mean"):
        print("Returning Mean of Size: %d "%size)
        h1 = (np.zeros(size**2).reshape(size,size) + 1) * (1/(size**2))
        return h1,[]
    else:
        print("This Kernel Is Not Supported")
        return [],[]

kernelGenerator(5,"mean")

Returning Mean of Size: 5 


(array([[0.04, 0.04, 0.04, 0.04, 0.04],
        [0.04, 0.04, 0.04, 0.04, 0.04],
        [0.04, 0.04, 0.04, 0.04, 0.04],
        [0.04, 0.04, 0.04, 0.04, 0.04],
        [0.04, 0.04, 0.04, 0.04, 0.04]]),
 [])